In [1]:
import requests
import time
import pprint
import pickle
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from unicodedata import normalize
from matplotlib import pyplot
import seaborn as sns
from sklearn.preprocessing import Imputer
%matplotlib
%pylab inline
pylab.rcParams['figure.figsize'] = (20, 10)
from sklearn.preprocessing import LabelEncoder
from collections import Counter

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [38]:
#Leemos la tabla 1
tablin = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\1.INDIVIDUOS_SOCIOS_O_BAJAS_train.txt', 
                    sep=';', engine="python", decimal=',', encoding="latin1")  


In [40]:
tabla= tablin

In [41]:
tabla.columns= ['ID', 'TELEFONOFIJO', 'TELEFONOMOVIL', 'EMAIL', 'DIRECCION', 'IDIOMA', 'FALLECIDO', 'SEXO', 'ONETOONE', 'CODPOSTAL', 'FECHANACIMIENTO']

In [42]:
#Procesar las fechas (nos quedamos solo con el año para poder calcular la edad)
pr=tabla
pr= pr.loc[:,['ID', 'FECHANACIMIENTO']]
pr= pr.dropna(subset=['FECHANACIMIENTO'])
pr['FECHANACIMIENTO'] = pd.to_datetime(pr['FECHANACIMIENTO'], errors= 'coerce')
pr= pr.dropna()
pr['FECHANACIMIENTO']=pr['FECHANACIMIENTO'].map(lambda x: x.strftime('%Y'))

In [43]:
pr['ACTUAL']= 2019

In [44]:
pr['FECHANACIMIENTO']=pd.to_numeric(pr['FECHANACIMIENTO'])
pr['ACTUAL']=pd.to_numeric(pr['ACTUAL'])

In [45]:
#Calculamos la edad
pr['EDAD']= pr['ACTUAL']-pr['FECHANACIMIENTO']
del pr['ACTUAL']
del pr['FECHANACIMIENTO']

In [46]:
tabla= tabla.merge(pr, on='ID', how='outer')

In [47]:
tabla=tabla.fillna(0)

In [48]:
del tabla['FECHANACIMIENTO']

In [49]:
tabla= tabla.replace('SI', '1')
tabla= tabla.replace('NO', '0')
tabla= tabla.replace('N', '0')
tabla= tabla.replace('H', '0')
tabla= tabla.replace('M', '1')

In [50]:
labelencoder = LabelEncoder()
tabla['IDIOMA'] = labelencoder.fit_transform(tabla['IDIOMA'])

In [51]:
tabla['TELEFONOMOVIL']= tabla['TELEFONOMOVIL'].astype('int')
tabla['TELEFONOFIJO']= tabla['TELEFONOFIJO'].astype('int')
tabla['EMAIL']= tabla['EMAIL'].astype('int')

In [52]:
#La nueva variable comunicaciones suma la cantidad de medios de comunicación disponibles para contactar a la persona
tabla['COMUNICACIONES'] = tabla['TELEFONOMOVIL']+tabla['EMAIL']+tabla['TELEFONOFIJO']

In [53]:
#Leemos la tabla 2
altas = pd.read_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\2.ALTASYBAJAS_train.txt', 
                    sep=';', engine="python", decimal=',', encoding="latin1")   

In [24]:
alta= altas
del alta['IDFAMILIA']
alta.columns = ['ID', 'FECHAALTA_alta', 'IDREGISTRO_alta', 'IDMEDIO_alta', 'IDCAMPANYA_alta']

In [25]:
tabla = tabla.merge(alta)

In [26]:
tabla=tabla.fillna(0)

In [27]:
tabla['FECHAALTA_alta'] = pd.to_datetime(tabla['FECHAALTA_alta'], format= "%d/%m/%Y %H:%M:%S")

In [29]:
#Formateamos la fecha de alta para que sea numérica y sólo contenga mes y año
tabla['year'] = tabla['FECHAALTA_alta'].dt.year
tabla['month'] = tabla['FECHAALTA_alta'].dt.month
tabla['month'] = tabla['month'].map("{:02}".format)
tabla['FECHAALTA_alta']=(tabla.year.astype(str) + tabla.month.astype(str))

In [30]:
del tabla['year']
del tabla['month']

In [32]:
tabla.to_csv(r'C:\Users\Marta\Desktop\MSF\datathon_train\procesamiento_1_2.csv', sep=';', encoding="latin1", decimal=',', index=False)

In [ ]:
#No usaremos la tabla mailings porque no aporta información relevante (propuesta de donativo y preferencia por correo o email, lo que ya tenemos en otra variable)